In [1]:
from utils import load_and_prepare_data
from neural_network import NeuralNetwork
import torch

dataset_path = './Dataset/http_ton.csv'

numerical_cols = [
        "duration",
        "dst_bytes",
        "missed_bytes",
        "src_bytes",
        "src_ip_bytes",
        "src_pkts",
        "dst_pkts",
        "dst_ip_bytes",
        "http_request_body_len",
        "http_response_body_len"

    ]

categorical_cols = [
        "proto",
        "conn_state",
        "http_status_code",
        "http_user_agent",
        "http_method"
    ]

target_col = 'type'
values_to_remove = {'type': ['mitm', 'dos']}

In [2]:
train_dataloader, valid_dataloader, test_dataloader, cat_cardinalities = load_and_prepare_data(
    file_path=dataset_path,
    target_col=target_col,
    numerical_cols=numerical_cols,
    categorical_cols=categorical_cols,
    rows_to_remove=values_to_remove
)



In [ ]:
model = NeuralNetwork(
    hidden_layers_sizes=[64, 32],
    cat_cardinalities=cat_cardinalities,
    embedding_dims=[min(50, (card + 1) // 2) for card in cat_cardinalities],
    num_numerical_features=len(numerical_cols),
    num_target_classes=6
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.fit(
    train_dataloader=train_dataloader,
    valid_dataloader=valid_dataloader,
    device=device,
)

--- Epoch: 0  |  F1 Score: 0.0442  |  Accuracy: 0.1144 ---


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x78f15d239c70>>
Traceback (most recent call last):
  File "/home/mat-rib/Documents/tabular_dnn/venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 800, in _clean_thread_parent_frames
    for identity in list(thread_to_parent_header.keys()):
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt: 


--- Epoch: 1  |  F1 Score: 0.0949  |  Accuracy: 0.1322 ---
